In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import pandas as pd
import numpy as np
import csv
from lxml import html
import requests
from langdetect import detect
import math

### STEP 1

#### STEP 1.1 -> Getting all the URLS

In [ ]:
# Needs to be changed to 201 to 301
outfile = open("test.txt", "w")
for i in range(201,301):
    
    base_url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)

    html = urlopen(base_url)
    soup = BeautifulSoup(html.read(), features="lxml")

    links = []
    regex = "/book/show/."

    for link in soup.find_all("a"):
        if re.match(regex, str(link.get("href"))):
            links.append("https://www.goodreads.com"+link.get("href"))
    
    #issues of duplicates        
    links = list(dict.fromkeys(links))
    
    for link in links:
        outfile.write(link+"\n")
outfile.close()

#### STEP 1.2 -> getting the HTML from the URLs, getting these HTMLs in different files

do not run the code below, this is just to show

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import lxml
import requests
import re
import os

# First, I make sure i am in the directory that has the text file with the URLS
path = "C:/Users/Guillaume/Desktop/ADM_HM3"
os.chdir(path)

i = 1
page = 1

with open("test_urls.txt", "r") as urls:
    # I previously created the 100 directories, from page 201 to page 300
    # I change the working directory for which i want the HTMLs 
    path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_201"
    os.chdir(path)

    for url in urls.readlines()[0:]:

        html = urlopen(url)

        soup = BeautifulSoup(html.read(), 'html.parser')
        
        # The logig here is not the best but it works- 
        # Basically, once every 100 urls treated I chance the directory
        
        if (i%101) != 0:
                # We stay in the current directory
            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

            i = i + 1

        else :

            page = page + 1
            path = "C:/Users/Guillaume/Desktop/ADM_HM3/page_20"+str(page)
            os.chdir(path)

            with open("article_"+str(i)+".html", "w", encoding="utf-8") as outfile:
                outfile.write(str(soup))

This has been done locally

#### STEP 1.3 -> Parsing the HTMLs files to retrieve all the required info

In [ ]:
# All functions to retrieve the informations

def get_title(soup):
    title = soup.find(id="bookTitle").contents[0].strip()
    return title

def get_series(soup):
    serie = soup.find(id="bookSeries").get_text().strip().replace("(","").replace(")","")
    regex = "#.*"
    serie = re.sub(regex, "",serie)
    return serie

def get_author(soup):
    author = soup.find_all(class_="authorName__container")
    author_list = " ".join([x.get_text().strip() for x in author])
    return author_list

def get_pages(soup):
    pages = soup.find(itemprop ="numberOfPages").get_text().split()[0]
    return int(pages)

def get_ratingValue(soup):
    ratingValue = soup.find(itemprop ="ratingValue").get_text().split()[0]
    return float(ratingValue)

def get_ratingCount(soup):
    test = []
    ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
    for x in ratingCount:
        test.append(x.get_text().split())
    return int(test[0][0].replace(",",""))
    
def get_reviewCount(soup):
    test = []
    ratingCount = soup.find_all(class_ ="gr-hyperlink", href="#other_reviews")
    for x in ratingCount:
        test.append(x.get_text().split())
    return int(test[1][0].replace(",",""))

def get_date(soup):
    regex = "by.*"
    date = soup.find_all(class_ = "row")
    date = re.sub(regex,"" ,str(date[1].contents[0]).replace("Published","")).strip()
    return date

def get_setting(soup):
    string = re.compile("/places/.")
    test = soup.find_all(href=string)
    
    places_list = " ".join([(str(a.contents[0])) for a in test])

    return places_list

def get_plot(soup):
    
    plot = soup.find(id="description")
    plot = plot.find_all(id= re.compile("freeText\.*"))

    final_plot = [] # It seems i need a list because joining on an empty string gives me a !"£$%&"@ blank
    
    # now there should not be a case where there are 3 span tags ...hopefully
    if len(plot) > 1:
        final_plot.append(plot[1].get_text())
    else:
        final_plot.append(plot[0].get_text())

    return final_plot[0]

def get_char(soup):
    string = re.compile("/characters/.")
    test = soup.find_all(href=string)

    char_list= " ".join([(str(a.contents[0])) for a in test])

    return char_list

def get_url(soup):

    regex = "https://www.goodreads.com/book/show/."
    ratingCount = soup.find(href =re.compile(regex), rel="canonical")
    return ratingCount.get('href')
    

In [ ]:
# We want 1 TSV file for 1 article
# In order to do this -> we first loop over the HTML
# We check if the language is indeed english
# for each HTML we create a new TSV and input the required DATA

i = 1

while i <= 4:
    html = open("article_"+str(i)+".html", "r", encoding="utf-8")
    soup = BeautifulSoup(html.read(), 'html.parser')
        
    with open("html_tsv_test_"+str(i)+".tsv", 'w', encoding="utf-8") as f_output:
        
        # We check the language of the splot
        if detect(get_plot(soup)) == "en":
            
            test_list_tsv = [get_title(soup), 
                             get_series(soup), 
                             get_author(soup),
                             get_ratingValue(soup),
                             get_ratingCount(soup), 
                             get_reviewCount(soup),
                             get_plot(soup),
                             get_pages(soup),
                             get_date(soup),
                             get_setting(soup),
                             get_char(soup),
                             get_url(soup)]
            
            header_list = ["bookTitle","bookSeries","bookAuthors","ratingValue","ratingCount","reviewCount","Plot","NumberofPages","Date","Characters","Setting","URL"]

            tsv_output = csv.writer(f_output, delimiter='\t')
            tsv_output.writerow(header_list)
            tsv_output.writerow(test_list_tsv)
            
            i = i + 1


## Step 2.0

Preprocessing
- Removing stopwords
- Removing punctuation
- Stemming
- Anything else you think it's needed ?

What is "anything else"?
 - lower, upper case ?

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [70]:
# We do the preprocessing and the creation of the vocabulary in one go
word_list = []

# start looping over the tsv files
for x in range(1,5):

    data = pd.read_csv("html_tsv_test_"+str(x)+".tsv", sep = "\t", encoding = "utf-8")
    plot = data['Plot'].item()
    
    #tokenisation
    word_tokens = word_tokenize(plot) 
    # stopwords
    stop_words = set(stopwords.words('english'))
    filter_stopsword = [w for w in word_tokens if not w in stop_words]
    # punctuations
    filter_punc = [w for w in filter_stopsword if w.isalnum()]
    # stemming
    ps = PorterStemmer() 
    filter_stem = [ps.stem(w) for w in filter_punc]
    # single string
    clean_plot = " ".join(filter_stem)
    
    # THIS PART TO REPLACE THE OLD PLOT
    # replace the old plot with the new one
    data["Plot"] = data["Plot"].replace(data['Plot'].item(),clean_plot)
    # ideally we will save over the old file
    data.to_csv("article_"+str(x)+".tsv", sep = '\t')
    
    # THIS PART TO CREATE THE VOCABULARY
    word_list.append(filter_stem)
    

In [71]:
# the Vocabulary is a list of list, we flatten that shit
flat_vocabulary = [item for sublist in word_list for item in sublist]
# putting the elements in a dico will eliminate the duplicates
vocabulary = dict.fromkeys(flat_vocabulary)

# We get the "integers" we are asked
i = 0
for k,v in vocabulary.items():
    vocabulary[k] = i
    i = i +1

vocabulary

{'A': 0,
 'power': 1,
 'novel': 2,
 'spiritu': 3,
 'bond': 4,
 'man': 5,
 'anim': 6,
 'there': 7,
 'door': 8,
 'end': 9,
 'silent': 10,
 'corridor': 11,
 'and': 12,
 'haunt': 13,
 'harri': 14,
 'pottter': 15,
 'dream': 16,
 'whi': 17,
 'els': 18,
 'would': 19,
 'wake': 20,
 'middl': 21,
 'night': 22,
 'scream': 23,
 'terror': 24,
 'lot': 25,
 'mind': 26,
 'fifth': 27,
 'year': 28,
 'hogwart': 29,
 'defens': 30,
 'against': 31,
 'dark': 32,
 'art': 33,
 'teacher': 34,
 'person': 35,
 'like': 36,
 'poison': 37,
 'honey': 38,
 'big': 39,
 'surpris': 40,
 'gryffindor': 41,
 'quidditch': 42,
 'team': 43,
 'loom': 44,
 'ordinari': 45,
 'wizard': 46,
 'level': 47,
 'exam': 48,
 'but': 49,
 'thing': 50,
 'pale': 51,
 'next': 52,
 'grow': 53,
 'threat': 54,
 'neither': 55,
 'magic': 56,
 'govern': 57,
 'author': 58,
 'grasp': 59,
 'tighten': 60,
 'must': 61,
 'discov': 62,
 'true': 63,
 'depth': 64,
 'strength': 65,
 'friend': 66,
 'import': 67,
 'boundless': 68,
 'loyalti': 69,
 'shock': 70,
 

### 2.1. Conjunctive query

#### 2.1.1 Create your index

In order to create this index we must loop for EACH word in ALLLLLLLL the tsv files we have. This will take time and undoubtedly crash a few times. 

We start this horrible looping.
For each word in the vocabulary -> we loop through each plot -> if the word is in that plot we retrieve we get the "document_id"
"where document_i is the id of a document that contains the word."

now...do documents have ids? -> We MUST align the article_id.tsv with the document_id in this dico

In [72]:
# We now create the inverted index : the format is as follow
#{ term_id_1:[document_1, document_2, document_4],
# term_id_2:[document_1, document_3, document_5, document_6], ....}

index = {}

for k,v in vocabulary.items():
    # the 5 needs to change to the number of articles + 1
    for x in range(1,5):
        with open("article_"+str(x)+".tsv", "r") as article:
            data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
            # this if statement needs to be changed to something like in 2.2
            if k in data["Plot"].item():
                if v not in index:
                    # Now i dont fucking get why we need these fucking integers when words are perfectly fine
                    index[v] = ["article_"+str(x)]
                elif v in index:
                    index[v].append("article_"+str(x))
print(index)

{0: ['article_1'], 1: ['article_1'], 2: ['article_1'], 3: ['article_1'], 4: ['article_1'], 5: ['article_1', 'article_3'], 6: ['article_1'], 7: ['article_2'], 8: ['article_2'], 9: ['article_2', 'article_3'], 10: ['article_2'], 11: ['article_2'], 12: ['article_2'], 13: ['article_2'], 14: ['article_2', 'article_4'], 15: ['article_2'], 16: ['article_2', 'article_4'], 17: ['article_2'], 18: ['article_2'], 19: ['article_2'], 20: ['article_2'], 21: ['article_2'], 22: ['article_2'], 23: ['article_2'], 24: ['article_2'], 25: ['article_2'], 26: ['article_2'], 27: ['article_2'], 28: ['article_2', 'article_4'], 29: ['article_2', 'article_4'], 30: ['article_2'], 31: ['article_2'], 32: ['article_2'], 33: ['article_2', 'article_3', 'article_4'], 34: ['article_2'], 35: ['article_2'], 36: ['article_2'], 37: ['article_2'], 38: ['article_2'], 39: ['article_2'], 40: ['article_2'], 41: ['article_2'], 42: ['article_2', 'article_4'], 43: ['article_2'], 44: ['article_2'], 45: ['article_2'], 46: ['article_2', 

In [73]:
# This needs to be changed to a "search engine" system. 

# We ask the user for input
query = input()
# we split each word of the query
query_words = query.split()

# We map the word of the query to their integers.
word_to_int = []
for word in query_words:
    # will have to check if that "in" works as intended
    if word in vocabulary:
        word_to_int.append(vocabulary[word])

# We ONLY want the articles for which ALL query words are present in the plot

sets = []
for y in word_to_int:
    # Does 12 in 120 return true?
    if y in index:
        sets.append(set(index[y]))

# We look for the intersection of articles
sets = [*set.intersection(*sets)]

harri year
[14, 28]


['article_2', 'article_4']

### 2.2 Conjunctive query & Ranking score

#### 2.2.1 Inverted index

In [ ]:
# dfi is the count of documents where the term i is present
# We initiate a new dico, keeping the word and starting the count at 0
dfi_dico = {}

for k,v in vocabulary.items():
    dfi_dico[k] = 0

In [ ]:
# The logic is as followed. For each word in the dictionary, open all the documents, look if it appears at least once.
# don't exactly remember why the dico "fuckthis" is needed. 
fuckthis= {}

for k, v in dfi_dico.items():
    dfi = 0
    # This needs to be replaced to the nb of articles + 1
    for x in range(1,5):
        with open("article_"+str(x)+".tsv", "r") as article:
            data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
            
        
            if (data["Plot"].item().split().count(k)) > 0:
                dfi += 1
    fuckthis[k] = dfi

In [63]:
# We the reason I want this format -> a list of lists with tuples is because 
# the code to get the tfidf was created with a library but the it turned out I cannot use this library
# so i created a code that would have a similar structure.

# finfin because I was tired
finfin = []
# finfin[0] -> the first article
# finfin[0][1] -> the tupple of the word-integer and its tdidf score
# finfin[0][1][1] -> the tdidf score for that specific int in that specific article

N = 4 # total number of plots/documents
# needs to be changed to the nb of articles - 1  or a different loop i dont care
for x in range(1,5):
    
    # tupi for cute tupple. 
    tupi = []
    
    with open("article_"+str(x)+".tsv", "r") as article:
        data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
        
        for x in data["Plot"].item().split():
            
            # frequence of the term i in the document j
            tfi = data["Plot"].item().count(x)
            
            # Weight of Term i in document j - wij
            wij = (1/tfi) * math.log(N/fuckthis[x])
            
            tupi.append((vocabulary[x], wij))
    finfin.append((tupi))
finfin

[[(0, 1.3862943611198906),
  (1, 1.3862943611198906),
  (2, 1.3862943611198906),
  (3, 1.3862943611198906),
  (4, 1.3862943611198906),
  (5, 1.3862943611198906),
  (6, 1.3862943611198906)],
 [(7, 1.3862943611198906),
  (8, 1.3862943611198906),
  (9, 0.46209812037329684),
  (10, 1.3862943611198906),
  (11, 1.3862943611198906),
  (12, 1.3862943611198906),
  (13, 1.3862943611198906),
  (14, 0.23104906018664842),
  (15, 1.3862943611198906),
  (16, 0.6931471805599453),
  (17, 1.3862943611198906),
  (18, 1.3862943611198906),
  (19, 1.3862943611198906),
  (20, 1.3862943611198906),
  (21, 1.3862943611198906),
  (22, 1.3862943611198906),
  (23, 1.3862943611198906),
  (24, 0.6931471805599453),
  (14, 0.23104906018664842),
  (25, 1.3862943611198906),
  (26, 1.3862943611198906),
  (27, 1.3862943611198906),
  (28, 0.6931471805599453),
  (29, 0.34657359027997264),
  (30, 1.3862943611198906),
  (31, 1.3862943611198906),
  (32, 0.6931471805599453),
  (33, 0.46209812037329684),
  (34, 1.386294361119890

In [ ]:
# The logic is the same as before
# to get the inverted index, we just check if the word of the dictionary appear in a plot
# if it does, we append the article and its tdidf score together.

index_2 = {}

for k,v in vocabulary.items():
    
    for x in range(1,5):
        
        with open("article_"+str(x)+".tsv", "r") as article:
            data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
            if k in ([x for x in data["Plot"].item().split()]):

                if v not in index_2:
                    # okej -> it's confusing but the only thing you need to understand is [y[0] for y in finfin[x-1]
                    # [y[0] for y in finfin[x-1] -> is just to get the right tupple, remember finfin
                    index_2[v] = [("article_"+str(x), finfin[x-1][[y[0] for y in finfin[x-1]].index(v)][1])]
                    
                elif v in index_2:
                    index_2[v].append(("article_"+str(x), finfin[x-1][[y[0] for y in finfin[x-1]].index(v)][1]))
print(index_2)

# Entirely hypothetical
Cosine Similary. 

We asume that A -> the first vector is simply the vector of TFIDF scores we calculed before
We assume that B -> the query vector is a vector of the same length of zeros except for the query words

A has the preprocessed text : "epic journey man animal" --> the TDIDF vector is: "0.12, 0.56, 0.10, 0.15"
B has the query : " -   journey -   animal" --> the TDIDF vector is: "0   , 0.56, 0   , 0.15"


In [76]:
# We could use the exact same code as before. this would not change anything.
# But i decided to use the inverted index created with the tdidf score


x = "harri year"
split_input = x.split()

word_to_int = []
for word in split_input:
    if word in vocabulary:
        word_to_int.append(vocabulary[word])
    


sets = []

for y in word_to_int:
    empty = []
    if y in index_2:
        for i in range(len(word_to_int)):
            empty.append(index_2[y][i][0])

    sets.append(set(empty))        

sets = [*set.intersection(*sets)]

So now I have the articles I need the following to perform the dot product:
- A
- B 
- ||A||
- ||B||

In [77]:
# the logic is as followed. For each article found. We get the its vector of tdidf
# then we uset


for x in sets:
    print(x)
    # finfin starts at 0 but article at 1
    x = x.replace("article_","")
    # get we alll the scores and transform into a vector
    A = [q[1] for q in finfin[int(x)-1]]
    A = np.array(A)

    len_a_tfidf = np.linalg.norm(A)

    indexes = []
    
    # For each word in the query, I want to replace the zero of the np.zeros vector 
    # Now, to replace the right 0, I need
    
    # Now it must be noted, it would be easier maybe to copy the vector A. and just replace by 0 for the some index.
    
    for i in word_to_int:
    
        A_index = [k[0] for k in finfin[int(x)-1]]
        the_index = A_index.index(i)
        indexes.append(finfin[int(x)-1][the_index])

    B = np.zeros(len(A))
    
    for j in indexes:
        B[j[0]] = j[1]

    len_b_tfidf = np.linalg.norm(B)

    Cossim = np.dot(A,B)/(len_a_tfidf*len_b_tfidf)
    print(Cossim)


article_2
0.1657716999054231
article_4
0.14185478949654853


In [ ]:
# Cannot use this library

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

list_plots = []
for x in range(1,5):
    with open("article_"+str(x)+".tsv", "r") as article:
        data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
        list_plots.append(data["Plot"].item())
        
tokenized_plots = [word_tokenize(doc) for doc in list_plots]

dictio = Dictionary(tokenized_plots)

corpus = [dictio.doc2bow(doc) for doc in tokenized_plots]


tfidf = TfidfModel(corpus)

tfidf[corpus[0]]

# Now take the first example -> the book "Jack"
# because we know from before that NO word in this plot will appear in the other plots all weights should be equal
#print(tfidf[corpus[1]])
#print("\n")
# to acess individual elements
#print(tfidf[corpus[1]][0])
#print("\n")
# To access the value of the tdidf score of the word
#print(tfidf[corpus[0]][0][1])

# We can use this to get a dictionary -> we could do this in the first part of this question
dictionary = dictio.token2id # we have to keep this indexing 

# now i have to build the invertex index again

index_2 = {}

for k,v in dictionary.items():
    for x in range(1,5):
        with open("article_"+str(x)+".tsv", "r") as article:
            data = pd.read_csv(article, sep = "\t", encoding = "utf-8")
            if k in ([x for x in data["Plot"].item().split()]):
                if v not in index_2:
                    # Now i dont fucking get why we need these fucking integers when words are perfectly fine
                    # [x-1] -> because the corpus starts at 0
                    # [v] -> because we want the v th element of the 
                    index_2[v] = [("article_"+str(x), tfidf[corpus[x-1]][[y[0] for y in tfidf[corpus[x-1]]].index(v)][1])]
                elif v in index_2:
                    index_2[v].append(("article_"+str(x), tfidf[corpus[x-1]][[y[0] for y in tfidf[corpus[x-1]]].index(v)][1]))

dictionary